<a href="https://colab.research.google.com/github/ushifali/final-23/blob/master/Cleaning_Facilities_Hotel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')
     

Mounted at /content/drive


In [41]:
import pandas as pd
import numpy as np
     
# read the second CSV file
hotels_df = pd.read_csv('/content/drive/MyDrive/Cleaned_Data/combined/final_hotel_dataset.csv', encoding='utf-8')


In [42]:
hotels_df

,Unnamed: 0,property_name,address,area,city,hotel_description,hotel_facilities,hotel_star_rating,image_urls,locality,latitude,longitude,pageurl,property_type,province,room_count,room_facilities,room_type,state,rating
0,0,007 Universe Resort,"Bollywood Boulevard, Khandpe, , Karjat",Karjat,Karjat,|A scenic village in the Konkan region Karjat ...,Banquet facilities|Business centre|Conference ...,3.0,//imghtlak.mmtcdn.com/images/hotels/2015090516...,NaN,18.876081,73.371433,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe,Maharasthra,4.00
1,1,007 Universe Resort-Karjat,"007 Universe Resort, Khandpe , Karjat",Karjat,Mumbai,|007 Universe Resort is a 4 star beautiful pro...,24 hours power back-up|Doctor on Call|Extra Ma...,4.0,//imghtlak.mmtcdn.com/images/hotels/2016011112...,NaN,18.876083,73.371436,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe,Kerala,3.20
2,2,1 and 2 bhk Suite apartments for rent,"Pilerne - Candolim Road, Pilerne Industrial Es...",North Goa,Goa,Service apartment for short term with all amme...,24 hours power back-up|Television|Microwave|Ho...,1.0,//imghtlak.mmtcdn.com/images/hotels/2015040510...,NaN,15.523210,73.787568,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel,NaN,NaN,NaN,Standard Room,Goa,4.50
3,3,"1 double a/c bedroom with bath, lounge, kitchen","Candolim , Goa",Goa,Goa,This property is on the ground floor apartment...,Activity centre|Conference Hall|Credit cards a...,1.0,http://www.tripvillas.com/media/origin/1532303...,NaN,15.519190,73.770790,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,1 Double Room Non Ac,Haryana,4.15
4,4,10 Blossoms Signature Hospitality Navrangpura,"43-SARDAR PATEL NAGAR,off CGrd , nr Girish Col...","nr Girish Cold Drinks,Nvrngpra",Ahmedabad,|Hotel Kalpadeep Inn is in proximity to famous...,Activity centre|Bird watching|Boating|Car rent...,3.0,//imghtlak.mmtcdn.com/images/hotels/2015042802...,NaN,23.034450,72.560501,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,`standard Room|Deluxe Room|Luxury Room,Delhi,3.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20596,20596,yogee hospitality &amp; accommodation,"shastri circal mdm hospital, Jodhpur",Ratanada,Jodhpur,The standard check-in time is 11:00 AM and the...,Airport Transfer Available / Surcharge|Beauty ...,0.0,//imghtlak.mmtcdn.com/images/hotels/2014071815...,Ratanada,28.613939,77.209021,http://www.goibibo.com/hotels/detail/Jodhpur/h...,Homestay,Jodhpur,NaN,Room Service |Air Conditioning |Balcony / Terr...,Deluxe AC,Rajasthan,6.20
20597,20597,young good luck houseboat,"GOLDEN DAL LAKE ,",Main Road,Srinagar,|less,Air conditioning|Banquet facilities|Bar|Doctor...,1.0,//imghtlak.mmtcdn.com/hotels/20151208111715813...,NaN,0.000000,0.000000,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,Deluxe,Goa,3.90
20598,20598,young queen of africa,"dal lake ,","CANAL ROAD, VESU",Srinagar,|less,Doctor on Call|Guide Service|Laundry Service|P...,1.0,//imghtlak.mmtcdn.com/images/hotels/2014112321...,NaN,0.000000,0.000000,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,`standard Room,Telangana,4.50
20599,20599,zahoora palace houseboat,"dal lake ,",Chanspa Road Leh,Srinagar,"|500 m from Bund Garden, 1.7 km from Gunjan Ci...",Activity Centre|Airport Transfer|Bonfire|Busin...,1.0,//imghtlak.mmtcdn.com/images/hotels/2009010813...,NaN,0.000000,0.000000,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel,NaN,NaN,NaN,`standard,KARNATAKA,4.70


In [ ]:
#taking rating from the hotel star rating 
# Extract the number using a regular expression
hotels_df['hotel_star_rating'] = hotels_df['hotel_star_rating'].str.extract(r'(\d+)').astype(float)

In [44]:
hotels_df['locality'] = hotels_df['locality'].fillna('')

In [ ]:
#need to remove duplicate hotel names
# Fill missing values by taking the values from the next row
hotels_df = hotels_df.fillna(method='bfill')

# Define a custom function to get the max room count
def max_room_count(x):
    x_numeric = pd.to_numeric(x, errors='coerce')
    x_max = np.nanmax(x_numeric)
    if np.isnan(x_max):
        return np.nan
    else:
        return str(x_max)
# Fill missing values by taking the values from the next row
hotels_df = hotels_df.fillna(method='bfill')


# Define a custom function to handle duplicate rows
def handle_duplicate_rows(df):
    # Group the DataFrame by the property_name column and aggregate the other columns
    agg_func = {
        'address': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0), # Use the longer string if it's a string column
        'area': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0), # Take the mean if it's a numeric column
        'city': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'hotel_description': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'hotel_facilities': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'hotel_star_rating': 'mean',
        'image_urls': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'locality': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'latitude': 'mean',
        'longitude': 'mean',
        'pageurl': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'property_type':lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'province': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'room_count': max_room_count,
        'room_facilities': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'room_type': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'state': lambda x: max(x, key=lambda v: len(str(v)) if isinstance(v, str) else 0),
        'rating': 'mean'
    }
    df = df.groupby('property_name').agg(agg_func).reset_index()
    return df

# Drop duplicate rows based on the property_name column
hotels_df = handle_duplicate_rows(hotels_df).drop_duplicates(subset=['property_name'])


In [ ]:
hotels_df

In [ ]:
# Replace missing or null values with an empty string
hotels_df['hotel_facilities'].fillna('', inplace=True)

#Define a lambda function to remove duplicates and count them
def remove_duplicates_and_count(x):
  facilities = x.split(', ')
  num_duplicates = len(facilities) - len(set(facilities))
  unique_facilities = set(facilities)
  return ', '.join(unique_facilities), num_duplicates

#Apply the lambda function to the "facilities" column
hotels_df[['hotel_facilities', 'Num_duplicates']] = hotels_df['hotel_facilities'].apply(remove_duplicates_and_count).apply(pd.Series)

#Print the updated DataFrame
print(hotels_df)

In [ ]:

# Print the values in the "Num_duplicates" column only if they are not equal to zero
print(hotels_df.loc[hotels_df['Num_duplicates'] != 0, 'Num_duplicates'])

In [ ]:
# Delete the "Num_duplicates" column
hotels_df.drop(columns=['Num_duplicates'], inplace=True)

# Save the updated DataFrame to a CSV file in your Google Drive
hotels_df.to_csv('/content/drive/MyDrive/Cleaned_Data/combined/final_hotel_dataset.csv', encoding='utf-8')

In [ ]:
#also this has cleaned data. i ran it again so erroe